# <center><h1>Companhia Aberta Demonstrativo Financeiro</h1></center>

# <h2>Load Libraries</h2>

In [1]:
import pandas as pd
import numpy as np

#<h2>Import Data</h2>

<h3>Load .csv</h3>

In [31]:
cia_aberta_df = pd.read_csv(r'dfp_cia_aberta_BPA_con_2020.csv', encoding='ISO-8859-1', sep=";")    # leio o .csv com os dados de todas as companhias abertas

<h3>Select Company</h3>

In [32]:
cia_aberta_df = cia_aberta_df[cia_aberta_df['CNPJ_CIA'] == '97.837.181/0001-47']    # dataframe = linhas do dataframe onde ('CNPJ_CIA' == '97.837.181/0001-47') == True
                                                                                    # (seleciono somente as linhas relativas a uma companhia de interesse)

In [148]:
df = cia_aberta_df[['ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']].copy()
df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']                                         # seleciono somente o último ou penúltimo exercício

In [149]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
56646,ÚLTIMO,1,Ativo Total,11498520.0
56648,ÚLTIMO,1.01,Ativo Circulante,4220022.0
56650,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,1.01.02,Aplicações Financeiras,0.0
56654,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...
56788,ÚLTIMO,1.02.04.02.07,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,1.02.04.02.08,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


#<h2>Wrangling</h2>

# <h3>Split CD_CONTA</h3>

In [150]:
df['CD_CONTA'].sort_values(ascending=True)

,CD_CONTA
56646,1
56648,1.01
56650,1.01.01
56652,1.01.02
56654,1.01.02.01
...,...
56788,1.02.04.02.07
56790,1.02.04.02.08
56792,1.02.04.02.09
56794,1.02.04.02.10


In [151]:
cd_conta_split = [string.split('.') for string in df['CD_CONTA']]

In [152]:
cd_conta_len = [len(lst) for lst in cd_conta_split]

In [153]:
num_levels = max(cd_conta_len)

In [154]:
num_levels

5

ROUND 1

JOIN BASATA SULLA LISTA DI LISTE CON I VALORI DEI CODICI

In [155]:
cd_conta_split

[['1'],
 ['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01

In [156]:
first_column = [row[0] for row in cd_conta_split]       # row[0] for row in cd_conta_split?

In [157]:
CD_DS_CONTA_dict = {cd: ds for cd, ds in zip(df['CD_CONTA'], df['DS_CONTA'])}

In [44]:
#key = first_column[0]
#key

In [43]:
#CD_DS_CONTA_dict[key]

In [158]:
def join(key):
    ds_conta = CD_DS_CONTA_dict[key]
    return ds_conta

In [159]:
ds_conta_1 = [join(x) for x in first_column]

In [160]:
df.insert(2, column='DS_CONTA_1', value=ds_conta_1)

In [161]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
56646,ÚLTIMO,1,Ativo Total,Ativo Total,11498520.0
56648,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
56650,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
56654,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...,...
56788,ÚLTIMO,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


Drop Rows

2 Strategie:
- 1. Se esiste almeno un CD_CONTA che inizia per '1', oltre al primo, eliminare il primo
- 2. Filtro con due condizioni: CLASS == DS_CONTA per quella riga e il valore della riga sucessiva per CLASS è uguale a quello precedente.
Esempio: 'Ativo Total' (CLASS_1) == 'Ativo Total' (DS_CONTA) e allo stesso tempo 'Ativo Total' == 'Ativo Total' (prima e seconda riga di CLASS_1)

In [112]:
cd_conta_split

[['1'],
 ['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01

cd_conta_split_t = ?

In [165]:
first_column = [row[0] for row in cd_conta_split]

In [166]:
second_column = [row[1] for row in cd_conta_split if len(row) > 1]

In [167]:
third_column = [row[2] for row in cd_conta_split if len(row) > 2]

...

In [162]:
cd_conta_split_t = [first_column, second_column, third_column]

In [163]:
cd_conta_split_t = []
for n in range(num_levels):
    cd_conta_split_t.append([row[n] for row in cd_conta_split if len(row) > n])

In [164]:
for row in cd_conta_split_t:
  print(f"{row}")

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02']
['01', '02', '02', '02', '02', '02', '02', '03', '03', '03', '03', '03', '03', '04', '05', '06', '06', '07', '08', '08', '08', '08', '01', '01', '01', '01', '01', '01',

In [168]:
cd_conta_split

[['1'],
 ['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01

Seleziona tutte le righe con un solo elemento:

In [169]:
cd_conta_len = [len(lst) for lst in cd_conta_split]

In [170]:
print(cd_conta_len)

[1, 2, 3, 3, 4, 5, 5, 4, 4, 3, 4, 5, 5, 4, 5, 3, 3, 3, 4, 3, 3, 4, 4, 4, 2, 3, 4, 5, 4, 4, 4, 5, 5, 4, 4, 4, 5, 4, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 4, 3, 4, 4, 4, 3, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [171]:
cd_conta_split[cd_conta_len==1]

['1']

In [172]:
cd_conta_len.index(1)

0

any(cd_conta_split sem elemento no index 0 [0] == '1'?)

sim -> cd_conta_split.__delitem__(0)

In [173]:
any(row[0] == '1' for row in cd_conta_split[1:])

True

In [174]:
cd_conta_split.__delitem__(0)

In [175]:
cd_conta_split

[['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01'],
 ['1

In [176]:
df = df.drop(df.index[0])

In [177]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
56648,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
56650,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
56654,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
...,...,...,...,...,...
56788,ÚLTIMO,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


ROUND 2

In [193]:
cd_conta_split

[['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01'],
 ['1

In [180]:
cd_conta_split_2 = [row[1:] for row in cd_conta_split]

In [181]:
cd_conta_split_2

[['01'],
 ['01', '01'],
 ['01', '02'],
 ['01', '02', '01'],
 ['01', '02', '01', '01'],
 ['01', '02', '01', '02'],
 ['01', '02', '02'],
 ['01', '02', '03'],
 ['01', '03'],
 ['01', '03', '01'],
 ['01', '03', '01', '01'],
 ['01', '03', '01', '02'],
 ['01', '03', '02'],
 ['01', '03', '02', '01'],
 ['01', '04'],
 ['01', '05'],
 ['01', '06'],
 ['01', '06', '01'],
 ['01', '07'],
 ['01', '08'],
 ['01', '08', '01'],
 ['01', '08', '02'],
 ['01', '08', '03'],
 ['02'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '01', '01'],
 ['02', '01', '02'],
 ['02', '01', '03'],
 ['02', '01', '04'],
 ['02', '01', '04', '01'],
 ['02', '01', '04', '02'],
 ['02', '01', '05'],
 ['02', '01', '06'],
 ['02', '01', '07'],
 ['02', '01', '07', '01'],
 ['02', '01', '08'],
 ['02', '01', '09'],
 ['02', '01', '09', '01'],
 ['02', '01', '09', '03'],
 ['02', '01', '09', '04'],
 ['02', '01', '10'],
 ['02', '01', '10', '01'],
 ['02', '01', '10', '02'],
 ['02', '01', '10', '03'],
 ['02', '01', '10', '04'],
 ['02', '01', '1

In [182]:
num_levels -= 1

In [183]:
num_levels

4

In [195]:
[".".join(row) for row in cd_conta_split_2]

['01',
 '01.01',
 '01.02',
 '01.02.01',
 '01.02.01.01',
 '01.02.01.02',
 '01.02.02',
 '01.02.03',
 '01.03',
 '01.03.01',
 '01.03.01.01',
 '01.03.01.02',
 '01.03.02',
 '01.03.02.01',
 '01.04',
 '01.05',
 '01.06',
 '01.06.01',
 '01.07',
 '01.08',
 '01.08.01',
 '01.08.02',
 '01.08.03',
 '02',
 '02.01',
 '02.01.01',
 '02.01.01.01',
 '02.01.02',
 '02.01.03',
 '02.01.04',
 '02.01.04.01',
 '02.01.04.02',
 '02.01.05',
 '02.01.06',
 '02.01.07',
 '02.01.07.01',
 '02.01.08',
 '02.01.09',
 '02.01.09.01',
 '02.01.09.03',
 '02.01.09.04',
 '02.01.10',
 '02.01.10.01',
 '02.01.10.02',
 '02.01.10.03',
 '02.01.10.04',
 '02.01.10.05',
 '02.01.10.06',
 '02.02',
 '02.02.01',
 '02.02.01.01',
 '02.02.01.04',
 '02.02.01.05',
 '02.02.02',
 '02.03',
 '02.03.01',
 '02.03.02',
 '02.03.03',
 '02.04',
 '02.04.01',
 '02.04.01.01',
 '02.04.01.02',
 '02.04.01.03',
 '02.04.02',
 '02.04.02.01',
 '02.04.02.02',
 '02.04.02.03',
 '02.04.02.04',
 '02.04.02.05',
 '02.04.02.06',
 '02.04.02.07',
 '02.04.02.08',
 '02.04.02.09',


In [196]:
df['CD_CONTA'] = [".".join(row) for row in cd_conta_split_2]

In [197]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
56648,ÚLTIMO,01,Ativo Total,Ativo Circulante,4220022.0
56650,ÚLTIMO,01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,01.02,Ativo Total,Aplicações Financeiras,0.0
56654,ÚLTIMO,01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,01.02.01.01,Ativo Total,Títulos para Negociação,0.0
...,...,...,...,...,...
56788,ÚLTIMO,02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


In [198]:
second_column = [row[0] for row in cd_conta_split_2]

In [199]:
print(second_column)

['01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02']


In [200]:
CD_DS_CONTA_dict = {cd: ds for cd, ds in zip(df['CD_CONTA'], df['DS_CONTA'])}

In [201]:
def join(key):
    ds_conta = CD_DS_CONTA_dict[key]
    return ds_conta

In [202]:
ds_conta_2 = [join(x) for x in second_column]

In [205]:
df.insert(3, column='DS_CONTA_2', value=ds_conta_2)

In [206]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
56648,ÚLTIMO,01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
56650,ÚLTIMO,01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
56654,ÚLTIMO,01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
...,...,...,...,...,...,...
56788,ÚLTIMO,02.04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,02.04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,02.04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,02.04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0
